# GP regression

An interesting application of GPs is for solving regression problems. This is usually referred to as **GP regression** (GPR) or less commonly as **Kriging**. The package `gpytorch` is used for a demonstration of GPR. We mainly follow the examples in the official documentation. In particular, a very simple regression problem based on a sine function is considered. As a first step, **hyperparameter estimation** of the GP parameters is performed. Then, the corresponding **posterior predictions** can be computed.

In [ ]:
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import torch
import gpytorch

from utils.modules import ExactInferenceGP

In [ ]:
torch.set_default_dtype(torch.float64)

## Data generation

In [ ]:
def f(x):
    '''Calculate ground truth.'''
    y = torch.sin(2 * torch.pi * x)
    return y

In [ ]:
num_samples = 10
noise_std = 0.1

x_train = torch.rand(num_samples)
y_train = f(x_train) + noise_std * torch.randn_like(x_train)

In [ ]:
x_values = torch.linspace(0, 1, 1001)
y_values = f(x_values)

fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(x_values.numpy(), y_values.numpy(), alpha=0.8, label='true function')
ax.scatter(x_train, y_train, alpha=0.8, label='training data')
ax.set(xlabel='x', ylabel='y')
ax.set_xlim((0, 1))
ax.legend()
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

## Hyperparameter estimation

In [ ]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()

model = ExactInferenceGP(
    x_train,
    y_train,
    likelihood,
    mean='zero'
)

mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [ ]:
num_iterations = 100

model.train() # train mode for (prior) hyperparameter estimation
likelihood.train()

for idx in range(num_iterations):
    optimizer.zero_grad()

    prior_mvn = model(x_train) # compute (prior) distribution in train mode
    loss = -mll(prior_mvn, y_train) # compute negative log-likelihood loss

    loss.backward()
    optimizer.step()

    if (idx + 1) % 2 == 0:
        print('Iteration {:d}, loss: {:.4f}'.format(idx + 1, loss.item()))

print('\nPrior std.: {:.4f}'.format(torch.sqrt(model.cov_module.outputscale).item()))
print('Lengthscale: {:.4f}'.format(model.cov_module.base_kernel.lengthscale.item()))
print('Noise std.: {:.4f}'.format(torch.sqrt(model.likelihood.noise).item()))

## Posterior predictions

In [ ]:
model.eval() # eval mode for posterior predictions
likelihood.eval()

with torch.no_grad():
    post_mvn = model(x_values) # compute posterior predictions in eval mode
    pred_mvn = likelihood(post_mvn) # include likelihood data model

    pred_mean = pred_mvn.mean
    pred_std = pred_mvn.stddev
    pred_var = pred_mvn.variance
    pred_cov = pred_mvn.covariance_matrix
    pred_lower, pred_upper = pred_mvn.confidence_region()

    pred_samples = post_mvn.sample(sample_shape=torch.Size((100,)))

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

true_function = ax.plot(x_values.numpy(), y_values.numpy(), alpha=0.8, zorder=3)
training_data = ax.scatter(x_train.numpy(), y_train.numpy(), alpha=0.8, zorder=4)

predictions = ax.plot(x_values.numpy(), pred_mean.numpy(), alpha=0.8, zorder=5)
uncertainty = ax.fill_between(x_values.numpy(), pred_lower.numpy(), pred_upper.numpy(), alpha=0.2, zorder=2)

ax.set_xlim((0, 1))
ax.legend((true_function[0], training_data, predictions[0], uncertainty),
          ('true function', 'training data', 'predictions', 'uncertainty'))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.plot(x_values.numpy(), pred_samples.T.numpy(), alpha=0.2)
ax.set_xlim((0, 1))
ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
ax.set_axisbelow(True)
fig.tight_layout()